In [ ]:
# 데이터 구조
import pandas as pd
# 행렬 연산
import numpy as np
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
# 데이터 분할:train, test
from sklearn.model_selection import train_test_split
# 회귀분석
from statsmodels.formula.api import ols
# qqplot, 회귀분석의 상수항 추가
from statsmodels.api import qqplot, add_constant
# 선형 모델 formula(y ~ X1 + X2 + ...)
import statsmodels.formula.api as smf
# 평가함수
from statsmodels.tools.eval_measures import rmse
# 결정계수 평가지표
from sklearn.metrics import r2_score
# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# 한글 맑은 고딕 적용
matplotlib.rc("font", family = "Malgun Gothic")
# 음수값 표시
matplotlib.rc("axes", unicode_minus = False)

In [ ]:
# 파일명, 변수, 값 등에 한글 포함시 engine="python" 지정
df_raw = pd.read_csv("../data/body.csv")
df_raw.head()

In [ ]:
# 산점도 행렬
sns.pairplot(df_raw, y_vars="FAT", x_vars=["WEIGHT", "NECK", "CHEST", "ABDOMEN"])
# 회귀 적합선 추가 : kind ="reg"
sns.pairplot(df_raw, y_vars="FAT", x_vars=["WEIGHT", "NECK", "CHEST", "ABDOMEN"], kind="reg")

In [ ]:
# 산점도 행렬
sns.pairplot(df_raw)

In [ ]:
# 상관관계 분석
df_raw.corr().round(3)

In [ ]:
# 학습용/평가용 데이터 분리
df_train, df_test = train_test_split(df_raw, test_size = 0.3, random_state = 1234)
print("학습용 데이터 크기 : {}".format(df_train.shape))
print("평가용 데이터 크기 : {}".format(df_test.shape))

In [ ]:
# 선형 회귀 분석
reg_model = smf.ols(formula = "FAT ~ AGE + WEIGHT + HEIGHT + NECK + CHEST + ABDOMEN + HIP + THIGH + KNEE + ANKLE + BICEPS + FOREARM + WRIST", data = df_train)
# 적합
reg_result = reg_model.fit()
print(reg_result.summary())

In [ ]:
# 선형 회귀 분석(유의하지 않은 변수 제거)
reg_model = smf.ols(formula = "FAT ~ WEIGHT + ABDOMEN + FOREARM + WRIST", data = df_train)
# 적합
reg_result = reg_model.fit()
print(reg_result.summary())

In [ ]:
# 설명변수 간의 다중공선성 확인
df_raw_x = df_train[['WEIGHT','ABDOMEN', 'FOREARM', 'WRIST']]
# statsmodels의 상수항 추가 함수 적용
df_raw_x_const = add_constant(df_raw_x)
# DataFrame으로 저장
df_vif = pd.DataFrame()
df_vif["variable"] = df_raw_x_const.columns
df_vif["VIF"] = [variance_inflation_factor(df_raw_x_const.values, i) for i in range(df_raw_x_const.shape[1])]
# VIF 정렬
df_vif.sort_values("VIF", inplace = True)
df_vif.round(3)

In [ ]:
# 잔차의 등분산성 : X-예측값, Y-잔차
plt.scatter(reg_result.fittedvalues, reg_result.resid)
plt.xlabel("fitted value")
plt.ylabel("residual")
plt.axhline(y = 0, linestyle = "--")

In [ ]:
# 잔차의 독립성 : X-데이터순서, Y-잔차
plt.scatter(df_train.index, reg_result.resid, marker="o")
plt.xlabel("Data")
plt.ylabel("residual")
plt.axhline(y = 0, linestyle = "-")
plt.axhline(y = 3*np.std(reg_result.resid), linestyle = "--")
plt.axhline(y = -3*np.std(reg_result.resid), linestyle = "--")

In [ ]:
# 잔차 정규성 검정:Q-Q plot. s-standardized line
qqplot(reg_result.resid, line = "q")

In [ ]:
# 설명변수 중요도
df_reg_coef = pd.DataFrame({"Coef": reg_result.params.values[1:]}, index = df_raw_x.columns)
df_reg_coef.plot.barh(y = "Coef", legend = False)

In [ ]:
y_pred = reg_result.predict(df_test)
print('평가용 데이터의 설명력 :', r2_score(df_test['FAT'], y_pred))